In [15]:
import pickle
from pathlib import Path
from sklearn.metrics import classification_report

In [3]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/incontext_learning/results/icl_Meta-Llama-3.1-8B-Instruct-bnb-4bit/results_10.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [4]:
results

{'ground_truths': [['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise']

In [5]:
grounds = results['ground_truths']

In [6]:
preds = results['predictions']

In [7]:
len(grounds), len(preds)

(1776, 1775)

In [8]:
preds[1667]

['anger', 'fear', 'surprise']

In [9]:
grounds[1667]

['fear', 'surprise']

In [10]:
del grounds[1667]

In [11]:
import ast
import json
import numpy as np

all_labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

def read_json_preds(decoded_outputs):
    
    preds = []

    #for output in outputs_l:
    for i, prediction in enumerate(decoded_outputs):
        try:
            # Use json.loads to safely parse the JSON-like string
            parsed_prediction = json.loads(prediction)
            # Append the values of the parsed prediction to preds
            preds.append(parsed_prediction['utterance_emotions'])
            
        except json.JSONDecodeError as e:
            print(f"Error decoding prediction: {i}")
            
    preds = [ast.literal_eval(item) for item in preds]    
            
    return preds



def labels_to_binary_matrix(label_list, all_labels):
    binary_matrix = np.zeros((len(label_list), len(all_labels)))
    
    for i, labels in enumerate(label_list):
        for label in labels:
            if label in all_labels:
                binary_matrix[i][all_labels.index(label)] = 1
                
    return binary_matrix

def opposite(component_type):

    if component_type == "anger":
        return "surprise"
    elif component_type == "disgust":
        return "joy"
    elif component_type == "fear":
        return "sadness"
    elif component_type == "sadness":
        return "anger"
    elif component_type == "surprise":
        return "disgust"
    elif component_type == "joy":
        return "fear"
    elif component_type == "Neutral":
        return "sadness"
    

def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

def post_process_icl(grounds, preds):

    for i,(x,y) in enumerate(zip(grounds, preds)):
        
        if len(x) != len(y):
            
            preds[i] = harmonize_preds(x, y)

    true_matrix = labels_to_binary_matrix(grounds, all_labels)
    predicted_matrix = labels_to_binary_matrix(preds, all_labels)

    return true_matrix, predicted_matrix

In [12]:
true_matrix, predicted_matrix = post_process_icl(grounds, preds)

In [14]:
print(classification_report(true_matrix, predicted_matrix, target_names=all_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.589     0.616     0.602       614
    surprise      0.643     0.616     0.629       485
        fear      0.476     0.473     0.475       406
     disgust      0.243     0.318     0.276        85
     sadness      0.421     0.406     0.413       347
         joy      0.560     0.620     0.588       429
     neutral      0.169     0.078     0.106       129

   micro avg      0.527     0.526     0.527      2495
   macro avg      0.443     0.447     0.441      2495
weighted avg      0.519     0.526     0.521      2495
 samples avg      0.502     0.502     0.502      2495



In [16]:
classification_file = Path("/Utilisateurs/umushtaq/emotion_analysis_comics/incontext_learning/results/icl_Meta-Llama-3.1-8B-Instruct-bnb-4bit/") / f"classification_report_{10}.pickle"

with classification_file.open('wb') as fh:
    
    pickle.dump(classification_report(true_matrix, predicted_matrix, target_names=all_labels, output_dict=True), fh)